In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from tqdm import tqdm
import json
import gc
import os

In [7]:
from config import storage_dir

# Load Model

In [ ]:

model_name = "Qwen/Qwen3-30B-A3B"
model_storage_dir = os.path.join(storage_dir, "lm_sys", model_name.split("/")[-1])
if not os.path.exists(model_storage_dir):
    os.makedirs(model_storage_dir)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto", 
    cache_dir="/n/holylfs06/LABS/krajan_lab/Lab/cfang/hf_cache/"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

# Load dataset

In [ ]:
ds = load_dataset("lmsys/lmsys-chat-1m")


# Helper functions

In [6]:
def format_conversation_for_qwen(user_prompt):
    """Format conversation for Qwen3-30B-A3B input with enable_thinking=False"""
    formatted = tokenizer.apply_chat_template(
        user_prompt,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # <-- Set this flag
    )
    return formatted

In [ ]:
def process_dataset_batch(dataset, batch_size=1, max_samples=None, context_length=4096):
    """Process dataset in batches to manage memory and parallelize generation"""
    processed_conversations = []
    
    # Limit samples if specified
    if max_samples:
        dataset = dataset.select(range(min(max_samples, len(dataset))))
    
    for i in tqdm(range(0, len(dataset), batch_size), desc="Processing conversations"):
        batch = dataset[i:i+batch_size]
        user_prompts = []
        user_turns = []
        for conversation in batch['conversation']:
            user_prompt = conversation[0]
            assert user_prompt['role'] == 'user'
            formatted_prompt = format_conversation_for_qwen([user_prompt])
            user_prompts.append(formatted_prompt)
            user_turns.append(user_prompt)
        
        # Tokenize as a batch
        inputs = tokenizer(
            user_prompts,
            return_tensors="pt",
            padding_side='left',
            padding=True,
            truncation=True,
            max_length=context_length
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=500,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
            )
        
        # Decode only the generated part for each sample
        for idx in range(len(user_prompts)):
            input_len = inputs['input_ids'][idx].shape[0]
            generated_ids = outputs[idx][inputs['input_ids'][idx].shape[0]:]
            response = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            new_conversation = [user_turns[idx], {'role': 'assistant', 'content': response}]
            processed_conversations.append({'conversation': new_conversation})
        
        # Clear GPU memory periodically
        if i % 100 == 0:
            torch.cuda.empty_cache()
            gc.collect()
    
    return processed_conversations

# Generate conversations and save dataset

In [7]:
# Process the dataset
print("Starting dataset processing...")
processed_data = process_dataset_batch(
    ds['train'], 
    batch_size=5,
    max_samples=5  # Adjust based on your needs and compute resources
)

Starting dataset processing...


Processing conversations:   0%|          | 0/1 [00:00<?, ?it/s]

Processing conversations: 100%|██████████| 1/1 [08:43<00:00, 523.60s/it]


In [ ]:
# Create new dataset
new_dataset = Dataset.from_list(processed_data)
print(f"Created dataset with {len(new_dataset)} conversations")

# Save the processed dataset
new_dataset.save_to_disk("qwq_generated_dataset")

# Clear the generation model to free memory for training
del model
torch.cuda.empty_cache()
gc.collect()


# Supervised Finetuning

In [ ]:
# Prepare for SFT Training
def format_conversation_for_sft(example):
    """Format conversation for SFT training"""
    conversation = example['conversation']
    
    # Format as a single string for SFT
    formatted_text = ""
    for turn in conversation:
        if turn['role'] == 'user':
            formatted_text += f"<|im_start|>user\n{turn['content']}<|im_end|>\n"
        elif turn['role'] == 'assistant':
            formatted_text += f"<|im_start|>assistant\n{turn['content']}<|im_end|>\n"
    
    return {"text": formatted_text}


In [ ]:
# Format dataset for SFT
sft_dataset = new_dataset.map(format_conversation_for_sft, remove_columns=new_dataset.column_names)

# Load model for training (you might want to use a smaller model for SFT)
training_model_name = "Qwen/QwQ-32B"  # or use a smaller Qwen model
training_model = AutoModelForCausalLM.from_pretrained(
    training_model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    cache_dir="/n/holylfs06/LABS/krajan_lab/Lab/cfang/hf_cache/"
)

training_tokenizer = AutoTokenizer.from_pretrained(training_model_name)
if training_tokenizer.pad_token is None:
    training_tokenizer.pad_token = training_tokenizer.eos_token

# Training arguments optimized for QwQ
training_args = TrainingArguments(
    output_dir="./qwq-sft-output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,  # Adjust based on your GPU memory
    gradient_accumulation_steps=8,
    learning_rate=5e-6,  # Lower learning rate for large models
    warmup_steps=100,
    logging_steps=10,
    save_steps=500,
    evaluation_strategy="no",
    save_strategy="steps",
    load_best_model_at_end=False,
    report_to=None,
    remove_unused_columns=False,
    dataloader_num_workers=0,
    fp16=True,  # Use mixed precision
    gradient_checkpointing=True,  # Save memory
    optim="adamw_torch",
    lr_scheduler_type="cosine",
    max_grad_norm=1.0,
)

# Initialize SFT Trainer
trainer = SFTTrainer(
    model=training_model,
    args=training_args,
    train_dataset=sft_dataset,
    tokenizer=training_tokenizer,
    max_seq_length=4096,  # Adjust based on your needs
    packing=False,  # Keep conversations intact
    dataset_text_field="text",
)

# Start training
print("Starting SFT training...")
trainer.train()

# Save the final model
trainer.save_model("./qwq-sft-final")
training_tokenizer.save_pretrained("./qwq-sft-final")

print("Training completed!")